In [1]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
import numpy as np
import os
import sys
import shutil
import time
import math
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import pickle

from lfads_datasets import DualALMDataset
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA
print(os.getcwd())
upcwd = '..'
stimtypecwd = upcwd + '/stimtypes'
logcwd = upcwd + '/Analyses'
cwd = upcwd + '/SavedPreds'
filenames = ['../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC4/BAYLORGC4_2018_03_22.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC4/BAYLORGC4_2018_03_21.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC4/BAYLORGC4_2018_03_23.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC4/BAYLORGC4_2018_03_20.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC4/BAYLORGC4_2018_03_25.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC12/BAYLORGC12_2018_04_03.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC12/BAYLORGC12_2018_04_04.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC12/BAYLORGC12_2018_04_05.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC12/BAYLORGC12_2018_04_06.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC13/BAYLORGC13_2018_04_23.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC13/BAYLORGC13_2018_04_24.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC13/BAYLORGC13_2018_04_25.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC15/BAYLORGC15_2018_05_07.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC15/BAYLORGC15_2018_05_08.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC15/BAYLORGC15_2018_05_09.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC15/BAYLORGC15_2018_05_10.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC15/BAYLORGC15_2018_05_11.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC17/BAYLORGC17_2018_06_08.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC17/BAYLORGC17_2018_06_11.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC18/BAYLORGC18_2018_05_31.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC19/BAYLORGC19_2018_06_20.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC19/BAYLORGC19_2018_06_21.mat', '../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC19/BAYLORGC19_2018_06_22.mat']

savenames = ['BAYLORGC4_2018_03_22.mat', 'BAYLORGC4_2018_03_21.mat', 'BAYLORGC4_2018_03_23.mat', 'BAYLORGC4_2018_03_20.mat', 'BAYLORGC4_2018_03_25.mat', 'BAYLORGC12_2018_04_03.mat', 'BAYLORGC12_2018_04_04.mat', 'BAYLORGC12_2018_04_05.mat', 'BAYLORGC12_2018_04_06.mat', 'BAYLORGC13_2018_04_23.mat', 'BAYLORGC13_2018_04_24.mat', 'BAYLORGC13_2018_04_25.mat', 'BAYLORGC15_2018_05_07.mat', 'BAYLORGC15_2018_05_08.mat', 'BAYLORGC15_2018_05_09.mat', 'BAYLORGC15_2018_05_10.mat', 'BAYLORGC15_2018_05_11.mat', 'BAYLORGC17_2018_06_08.mat', 'BAYLORGC17_2018_06_11.mat', 'BAYLORGC18_2018_05_31.mat', 'BAYLORGC19_2018_06_20.mat', 'BAYLORGC19_2018_06_21.mat', 'BAYLORGC19_2018_06_22.mat']

trainset = False


for timebins in ['']:
  for trainstimtype in ['']:
    variance_explained = {}


    gate_types = ['no', '', 'TanhSquare_scalargateFromAll', 'FlipTanhSquare_scalargateFromAll', 
                  'MonoTanhSquare_scalargateFromAll','attractor_TanhSquare_scalargateFromAll', 
                  'attractor_FlipTanhSquare_scalargateFromAll', 'attractor_MonoTanhSquare_scalargateFromAll',
                 'attractor_nobridge', 'attractor_normalbridge']
    
    totvariance_explained = {}





    for bridge in gate_types:
        variance_explained[bridge] = []
        totvariance_explained[bridge] = []
        for indexX in range(len(filenames)):
          filename = filenames[indexX]
          savename = savenames[indexX]
          print(filename)
          tempvars = {0.001: [], 0.0001: [], 0.00001: [], 0.000001: []}
          tempvarsnum = {0.001: [], 0.0001: [], 0.00001: [], 0.000001: []}
          tempvarsdenom = {0.001: [], 0.0001: [], 0.00001: [], 0.000001: []}
          for wd in [0.001, 0.0001, 0.00001, 0.000001]:
            weight_decay = wd


            no_f = 'nof'
            print('BRIDGE', bridge)
            if bridge == '' or bridge == 'no':
                pred_file = 'X'+bridge + 'bridgeFullToCDPred'
            else:
                pred_file = 'OneStepModel' + str(1.0) + '_' + str(bridge) + '_' + str(timebins)+str(trainstimtype)+str(weight_decay)+ savename + '_'


            b_file = 'OneStepModel' + str(1.0) + '_' + str('attractor_normalbridge') + '_' + str(timebins)+str(trainstimtype)+str(weight_decay)+ savename + '_'#'XRegSuperSimple' + str(1.0) + '_' + '_' + str(timebins)+str(trainstimtype)+ savename + '_'


            train_baseline_L = np.load(cwd+'/'+b_file+'TRAIN_BASELINE_L.npy')
            train_baseline_R = np.load(cwd+'/'+b_file+'TRAIN_BASELINE_R.npy')
            test_baseline_L = np.load(cwd+'/'+b_file+'TEST_BASELINE_L.npy')
            test_baseline_R = np.load(cwd+'/'+b_file+'TEST_BASELINE_R.npy')
            if trainset:
              test_baseline_L = train_baseline_L
              test_baseline_R = train_baseline_R
                
            stimtype_file = 'XRegSuperSimple' + str(1.0) + '_' + '_' + str(timebins)+str(trainstimtype)+ savename + '_'
            train_stimtypes = np.load(stimtypecwd+'/'+stimtype_file + 'TRAIN_stimtypes.npy')
            #print('stst', train_stimtypes.shape, train_baseline_L.shape, train_baseline_R.shape)
            test_stimtypes = np.load(stimtypecwd+'/'+stimtype_file + 'TEST_stimtypes.npy')
            if trainset:
                test_stimtypes = train_stimtypes


            if bridge == '' or bridge == 'no':
                model_preds_train_L = np.squeeze(np.load(cwd+'/'+pred_file+'TRAINL'+timebins+trainstimtype+'.npy')[indexX])#[:, :, :]#[:, 13:, :]
                model_preds_train_R = np.squeeze(np.load(cwd+'/'+pred_file+'TRAINR'+timebins+trainstimtype+'.npy')[indexX])#[:, :, :]#[:, 13:, :]
            else:
                model_preds_train_L = np.load(cwd+'/'+pred_file+'TRAIN_L.npy')#[:, :, :]#[:, 13:, :]
                model_preds_train_R = np.load(cwd+'/'+pred_file+'TRAIN_R.npy')#[:, :, :]#[:, 13:, :]
            if trainset:
              model_preds_test_L = model_preds_train_L
              model_preds_test_R = model_preds_train_R
            else: 
                if bridge == '' or bridge == 'no':
                    model_preds_test_L = np.squeeze(np.load(cwd+'/'+pred_file+'TESTL'+timebins+trainstimtype+'.npy')[indexX])#[:, :, :]#[:, 13:, :]#model_preds_train_L#np.load(args.pred_file+'TEST_L.npy')#[:, :, :]#[:, 13:, :]
                    model_preds_test_R = np.squeeze(np.load(cwd+'/'+pred_file+'TESTR'+timebins+trainstimtype+'.npy')[indexX])#[:, :, :]#[:, 13:, :]#model_preds_train_R#np.load(args.pred_file+'TEST_R.npy')#[:, :, :]#[:, 13:, :]

                else:
                    model_preds_test_L = np.load(cwd+'/'+pred_file+'TEST_L.npy')#[:, :, :]#[:, 13:, :]
                    model_preds_test_R = np.load(cwd+'/'+pred_file+'TEST_R.npy')#[:, :, :]#[:, 13:, :]




 
            if model_preds_train_L.shape[0] < 0:
                continue



            train_data_L = np.load(cwd+'/'+b_file+'TRAIN_TRUTH_L.npy')# - train_baseline_L
            train_data_R = np.load(cwd+'/'+b_file+'TRAIN_TRUTH_R.npy')# - train_baseline_R

            if trainset:
                test_data_L = train_data_L
                test_data_R = train_data_R
            else:
                test_data_L = np.load(cwd+'/'+b_file+'TEST_TRUTH_L.npy')# - test_baseline_L
                test_data_R = np.load(cwd+'/'+b_file+'TEST_TRUTH_R.npy')# - test_baseline_R



            print(model_preds_train_L.shape, model_preds_train_R.shape, model_preds_test_L.shape, model_preds_test_R.shape)




            model_preds_trainX = [model_preds_train_L, model_preds_train_R]
            model_preds_testX = [model_preds_test_L, model_preds_test_R]
            train_dataX = [train_data_L, train_data_R]
            test_dataX = [test_data_L, test_data_R]
            train_baselineX = [train_baseline_L, train_baseline_R]
            test_baselineX = [test_baseline_L, test_baseline_R]
            locs = ['left_ALM', 'right_ALM']
            for leftright in [0, 1]:
              for stype in [[0,0], [1,2], [2,1], [3,3]]:
                if leftright == 0:
                    print('LEFTTTTTT')
                else:
                    print('RIGHTTTTT')
                
 
                numneurons = 1#model_preds_trainX[leftright].shape[1]
                if timebins == '':
                    model_preds_train = model_preds_trainX[leftright].reshape(8, -1, numneurons)
                    model_preds_test = model_preds_testX[leftright].reshape(8, -1, numneurons)
                    train_baseline = train_baselineX[leftright].reshape(8, -1, numneurons)
                    test_baseline = test_baselineX[leftright].reshape(8, -1, numneurons)
                    train_data = train_dataX[leftright].reshape(8, -1, numneurons)
                    test_data = test_dataX[leftright].reshape(8, -1, numneurons)
                else:
                    model_preds_train = model_preds_trainX[leftright].reshape(4, -1, numneurons)
                    model_preds_test = model_preds_testX[leftright].reshape(4, -1, numneurons)
                    train_baseline = train_baselineX[leftright].reshape(4, -1, numneurons)
                    test_baseline = test_baselineX[leftright].reshape(4, -1, numneurons)
                    train_data = train_dataX[leftright].reshape(4, -1, numneurons)
                    test_data = test_dataX[leftright].reshape(4, -1, numneurons)

                train_data = train_data[:, train_stimtypes == stype[leftright]].reshape(-1, numneurons)
                model_preds_train = model_preds_train[:, train_stimtypes == stype[leftright]].reshape(-1, numneurons)
                train_baseline = train_baseline[:, train_stimtypes == stype[leftright]].reshape(-1, numneurons)
                test_data = test_data[:, test_stimtypes == stype[leftright]].reshape(-1, numneurons)
                model_preds_test = model_preds_test[:, test_stimtypes == stype[leftright]].reshape(-1, numneurons)

                test_baseline = test_baseline[:, test_stimtypes == stype[leftright]].reshape(-1, numneurons)





                
                test_data_pred = model_preds_test
                train_data_pred = model_preds_train

     
                avg_se_per_trial = np.sum(np.square(test_data_pred - test_data))/model_preds_test.shape[0]

                
                total_avg_pred = test_baseline#np.mean(train_data, axis=0, keepdims=True) * 10
                total_avg_pred_se_per_trial = np.sum(np.square(total_avg_pred - test_data))/model_preds_test.shape[0]


                print('Variance Explained', 1 - avg_se_per_trial/total_avg_pred_se_per_trial)

                tempvars[wd].append(1 - avg_se_per_trial/total_avg_pred_se_per_trial)
                tempvarsnum[wd].append(avg_se_per_trial*test_baseline.shape[0])
                tempvarsdenom[wd].append(total_avg_pred_se_per_trial*test_baseline.shape[0])
          maxval = -999999999999999
          maxkey = 200
          for key in tempvars.keys():
              val = 1 - (np.sum(tempvarsnum[key]) / np.sum(tempvarsdenom[key])) 
              if val > maxval:
                  maxval = val
                  maxkey = key
          variance_explained[bridge].extend(tempvars[maxkey])
          totvariance_explained[bridge].append(maxval)
         



                

    name = logcwd+'/'+'OneStepModels_var_explained'+ str(timebins)+str(trainstimtype)+'.npy'





   
    if trainset:
        name = 'TRAIN' + name

    np.save(name, variance_explained)

    name = logcwd+'/'+'OneStepModels_totvar_explained'+ str(timebins)+str(trainstimtype)+'.npy'
   
    if trainset:
        name = 'TRAIN' + name


    np.save(name, totvariance_explained)
    





/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


/home/druckmannuser/dual_alm_analysis/GatingModels/Code
../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC4/BAYLORGC4_2018_03_22.mat
BRIDGE no
(2672,) (2672,) (688,) (688,)
LEFTTTTTT
Variance Explained -0.015759452348977465
LEFTTTTTT
Variance Explained 0.28165707228234127
LEFTTTTTT
Variance Explained -0.3630582291574218
LEFTTTTTT
Variance Explained 0.2780042024752817
RIGHTTTTT
Variance Explained 0.03636257328578085
RIGHTTTTT
Variance Explained -0.03425064760127383
RIGHTTTTT
Variance Explained 0.10107252550763479
RIGHTTTTT
Variance Explained 0.061646928982155846
BRIDGE no
(2672,) (2672,) (688,) (688,)
LEFTTTTTT
Variance Explained -0.015759452348977465
LEFTTTTTT
Variance Explained 0.28165707228234127
LEFTTTTTT
Variance Explained -0.3630582291574218
LEFTTTTTT
Variance Explained 0.2780042024752817
RIGHTTTTT
Variance Explained 0.03636257328578085
RIGHTTTTT
Variance Explained -0.03425064760127383
RIGHTTTTT
Variance Explained 0.10107252550763479
RIGHTTTTT
Variance Explained 0.06164692898

(1096,) (1096,) (272,) (272,)
LEFTTTTTT
Variance Explained 0.018676196405711054
LEFTTTTTT
Variance Explained 0.30298827880246404
LEFTTTTTT
Variance Explained 0.5139259661030863
LEFTTTTTT
Variance Explained -0.31512497509195336
RIGHTTTTT
Variance Explained 0.0464984051144085
RIGHTTTTT
Variance Explained 0.27184323055983306
RIGHTTTTT
Variance Explained -0.047151087564271554
RIGHTTTTT
Variance Explained 0.507598061710691
BRIDGE no
(1096,) (1096,) (272,) (272,)
LEFTTTTTT
Variance Explained 0.018676196405711054
LEFTTTTTT
Variance Explained 0.30298827880246404
LEFTTTTTT
Variance Explained 0.5139259661030863
LEFTTTTTT
Variance Explained -0.31512497509195336
RIGHTTTTT
Variance Explained 0.0464984051144085
RIGHTTTTT
Variance Explained 0.27184323055983306
RIGHTTTTT
Variance Explained -0.047151087564271554
RIGHTTTTT
Variance Explained 0.507598061710691
../../Data/DualALMRecordinbgsDataSet2018_04_14/BAYLORGC12/BAYLORGC12_2018_04_05.mat
BRIDGE no
(1592,) (1592,) (384,) (384,)
LEFTTTTTT
Variance Exp

(2024,) (2024,) (488,) (488,)
LEFTTTTTT
Variance Explained 0.0952904402865854
LEFTTTTTT
Variance Explained 0.24302944329415432
LEFTTTTTT
Variance Explained 0.05804969631107704
LEFTTTTTT
Variance Explained 0.16354738183629136
RIGHTTTTT
Variance Explained -0.34839556438985664
RIGHTTTTT
Variance Explained 0.6459347899448785
RIGHTTTTT
Variance Explained -0.20441972378792683
RIGHTTTTT
Variance Explained 0.8349762350962362
BRIDGE no
(2024,) (2024,) (488,) (488,)
LEFTTTTTT
Variance Explained 0.0952904402865854
LEFTTTTTT
Variance Explained 0.24302944329415432
LEFTTTTTT
Variance Explained 0.05804969631107704
LEFTTTTTT
Variance Explained 0.16354738183629136
RIGHTTTTT
Variance Explained -0.34839556438985664
RIGHTTTTT
Variance Explained 0.6459347899448785
RIGHTTTTT
Variance Explained -0.20441972378792683
RIGHTTTTT
Variance Explained 0.8349762350962362
BRIDGE no
(2024,) (2024,) (488,) (488,)
LEFTTTTTT
Variance Explained 0.0952904402865854
LEFTTTTTT
Variance Explained 0.24302944329415432
LEFTTTTTT
V

KeyboardInterrupt: 

In [7]:
import numpy as np
from scipy.stats import ttest_rel

totvar = np.load(logcwd+'/'+'OneStepModels_totvar_explained'+'.npy').tolist()
for bridge in gate_types:
    print(bridge, np.mean(totvar[bridge]))
    
ttest_rel(totvar['attractor_TanhSquare_scalargateFromAll'], totvar['TanhSquare_scalargateFromAll'])



no 0.1369457667132536
 0.1440035813260241
TanhSquare_scalargateFromAll 0.1546237545123432
FlipTanhSquare_scalargateFromAll 0.15381489056525055
MonoTanhSquare_scalargateFromAll 0.15220510307854546
attractor_TanhSquare_scalargateFromAll 0.16080449767701135
attractor_FlipTanhSquare_scalargateFromAll 0.15846336813879014
attractor_MonoTanhSquare_scalargateFromAll 0.1512604884723392
attractor_nobridge 0.13766135306033128
attractor_normalbridge 0.14502153071584567


Ttest_relResult(statistic=1.6434840720387838, pvalue=0.11449949128130214)

In [4]:
var = np.load(logcwd+'/'+'OneStepModels_totvar_explained'+'.npy').tolist()
for bridge in gate_types:
    vals = var[bridge]



    vals_n = vals[::4]
    vals_i = vals[1::4]
    vals_c = vals[2::4]
    vals_b = vals[3::4]
    print(bridge, np.mean(vals_n), np.mean(vals_i), np.mean(vals_c), np.mean(vals_b))

no 0.14602075856426758 0.13816171094536736 0.11598594682816676 0.14974842727560445
 0.1640665521718036 0.13845808958618136 0.11176430188560722 0.16526974172740017
TanhSquare_scalargateFromAll 0.16254209298646702 0.16398823439083374 0.12284785200843863 0.17201545549389144
FlipTanhSquare_scalargateFromAll 0.17078419737395215 0.15089411032511224 0.12793524802055975 0.16801222973660362
MonoTanhSquare_scalargateFromAll 0.16158653898449601 0.1604292147076677 0.12648077872806016 0.16194763525704045
attractor_TanhSquare_scalargateFromAll 0.17161450372148326 0.16582736457224717 0.13945259775192922 0.16742733005946064
attractor_FlipTanhSquare_scalargateFromAll 0.16982337334636513 0.16390305159686244 0.13096972379072927 0.17129611495768643
attractor_MonoTanhSquare_scalargateFromAll 0.16109990136457922 0.16332673137730838 0.12260131345215602 0.159364711539908
attractor_nobridge 0.14623296365135022 0.13900975299954413 0.11735376636915407 0.15012644445346585
attractor_normalbridge 0.1579288361800759